In [2]:
import py_d3

In [3]:
py_d3.load_ipython_extension(get_ipython())

### create sample data

In [4]:
import random
import pandas as pd

In [5]:
items = ['item@item{0:02d}'.format(i) for i in range(100)]

relations = []
for i in range(300):
    item1 = random.randint(0, len(items)-1)
    
    while True:
        item2 = random.randint(0, len(items)-1)
        if item1 != item2:
            break
    
    relations.append([items[item1], items[item2]])
    relations.append([items[item2], items[item1]])

In [6]:
samples = pd.DataFrame(relations)

In [7]:
samples = samples.sort_values(by=0)

### to json

In [8]:
unique_names = list(samples[0].unique())
pairs = pd.DataFrame(columns=['name', 'pairs'])

for name in unique_names:
    pairs_list = list(samples[samples[0] == name][1])
    df = pd.DataFrame([[name, pairs_list]], columns=['name', 'pairs'])
    pairs = pairs.append(df, ignore_index=True)


json_format = pairs.to_json(orient='records')
with open('./data/sample1.json', 'w') as f:
    f.write(json_format)

In [10]:
%%d3

<style>

.node {
  font: 300 11px "Helvetica Neue", Helvetica, Arial, sans-serif;
  fill: #bbb;
}

.node:hover {
  fill: #000;
}

.link {
  stroke: #80DEEA;
  stroke-opacity: .4;
  fill: none;
  pointer-events: none;
}

.node:hover,
.node--source,
.node--target {
  font-weight: 700;
}

.node--source {
  fill: #2ca02c;
}

.node--target {
  fill: #d62728;
}

.link--source,
.link--target {
  stroke-opacity: 1;
  stroke-width: 2px;
}

.link--source {
  stroke: #d62728;
}

.link--target {
  stroke: #d62728;
}

</style>
<div id="d3area"></div>

<script>

var diameter = 800,
    radius = diameter / 2,
    innerRadius = radius - 120;

var cluster = d3.layout.cluster()
    .size([360, innerRadius])
    .sort(null)
    .value(function(d) { return d.size; });

var bundle = d3.layout.bundle();

var line = d3.svg.line.radial()
    .interpolate("bundle")
    .tension(.85)
    .radius(function(d) { return d.y; })
    .angle(function(d) { return d.x / 180 * Math.PI; });

var svg = d3.select("#d3area").append("svg")
    .attr("width", diameter)
    .attr("height", diameter)
    .style("width", diameter + 80 + "px")
    .style("height", diameter + 80 + "px")
    .append("g")
    .attr("transform", "translate(" + radius + "," + radius + ")");

var link = svg.append("g").selectAll(".link"),
    node = svg.append("g").selectAll(".node");

d3.json("./data/sample1.json", function(error, classes) {
  if (error) throw error;

  var nodes = cluster.nodes(itemHierarchy(classes)),
      links = storeRelations(nodes);

  link = link
      .data(bundle(links))
    .enter().append("path")
      .each(function(d) { d.source = d[0], d.target = d[d.length - 1]; })
      .attr("class", "link")
      .attr("d", line);

  node = node
      .data(nodes.filter(function(n) { return !n.children; }))
    .enter().append("text")
      .attr("class", "node")
      .attr("dy", ".31em")
      .attr("transform", function(d) { return "rotate(" + (d.x - 90) + ")translate(" + (d.y + 8) + ",0)" + (d.x < 180 ? "" : "rotate(180)"); })
      .style("text-anchor", function(d) { return d.x < 180 ? "start" : "end"; })
      .text(function(d) { return d.key; })
      .on("mouseover", mouseovered)
      .on("mouseout", mouseouted);
});

function mouseovered(d) {
  node
      .each(function(n) { n.target = n.source = false; });

  link
      .classed("link--target", function(l) { if (l.target === d) return l.source.source = true; })
      .classed("link--source", function(l) { if (l.source === d) return l.target.target = true; })
    .filter(function(l) { return l.target === d || l.source === d; })
      .each(function() { this.parentNode.appendChild(this); });

  node
      .classed("node--target", function(n) { return n.target; })
      .classed("node--source", function(n) { return n.source; });
}

function mouseouted(d) {
  link
      .classed("link--target", false)
      .classed("link--source", false);

  node
      .classed("node--target", false)
      .classed("node--source", false);
}

d3.select(self.frameElement).style("height", diameter + "px");

function itemHierarchy(classes) {
  var map = {};

  function find(name, data) {
    var node = map[name], i;
    if (!node) {
      node = map[name] = data || {name: name, children: []};
      if (name.length) {
        node.parent = find(name.substring(0, i = name.lastIndexOf("@")));
        node.parent.children.push(node);
        node.key = name.substring(i + 1);
      }
    }
    return node;
  }

  classes.forEach(function(d) {
    find(d.name, d);
  });

  return map[""];
}

function storeRelations(nodes) {
  var map = {},
      pairs = [];

  nodes.forEach(function(d) {
    map[d.name] = d;
  });

  nodes.forEach(function(d) {
    if (d.pairs) d.pairs.forEach(function(i) {
      pairs.push({source: map[d.name], target: map[i]});
    });
  });

  return pairs;
}

</script>